In [1]:
'''
本文件作用为
给定原始kaggle数据 US_Accidents_March23.csv
生成清洗后的两张数据表 —— data_0601.csv、data_0601_one_hot.csv
这份代码用于验证，您也可以不运行，直接下载网盘中的三份数据放在本地
'''
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# 1.通过欠采样、去空值行、列选择生成data_0601.csv

In [2]:
df = pd.read_csv('US_Accidents_March23.csv')
df2 = df[(pd.to_datetime(df['Start_Time']).dt.year ==2022) 
                  | (pd.to_datetime(df['Start_Time']).dt.year ==2023)
                  |(df['Severity'] == 1)
                 |(df['Severity'] == 3)
                 |(df['Severity'] == 4)] # 欠采样
df2['year'] = df2['Start_Time'].apply(lambda x: pd.to_datetime(x).year)
df2['month'] = df2['Start_Time'].apply(lambda x: pd.to_datetime(x).month)
xy = df2[[ 'month','Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight','Severity']]
xy2 = xy.dropna(axis=0).reset_index()# 删除有空值的行
xy2.to_csv('data_0601.csv',index = False)

# 2.在第一份数据基础上，进行独热编码，生成第二份数据data_0601_one_hot.csv

In [3]:
category_cns = ['Wind_Direction','Weather_Condition','Sunrise_Sunset'
            ,'Civil_Twilight','Nautical_Twilight','Astronomical_Twilight','month']
new_cns = []
length = len(xy2)
xy3 = xy2.copy()
for str_cn in category_cns:
    d_types = list(xy2[str_cn].value_counts().keys())
    for value in d_types:
        value = str(value) # 防止特殊情况报错
        tmp = str_cn + '_' + value
        xy3[tmp] = 0
        new_cns.append(tmp)
    for i in tqdm(range(length)):
        xy3.loc[i, str_cn + '_' + str(xy3.loc[i,str_cn])] = 1
xy3 = xy3[[ 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Severity'] + new_cns]
xy3.to_csv('data_0601_one_hot.csv',index = False)

100%|██████████████████████████████████████████████████████████████████████| 2496090/2496090 [04:56<00:00, 8417.81it/s]
